In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [13]:
vix = pd.read_csv('VIX.csv')
vix.dropna(inplace=True)
vix['Close'] = vix['Adj Close']
vix.drop(['Adj Close'], axis=1, inplace=True)
vix = vix[vix['Date'] > '1993-01-28']

In [14]:
vix.tail(10)

,Date,Open,High,Low,Close,Volume
8983,2024-06-07,12.69,13.08,12.11,12.22,0.0
8984,2024-06-10,13.09,13.28,12.62,12.74,0.0
8985,2024-06-11,12.85,13.47,12.78,12.85,0.0
8986,2024-06-12,13.10,13.15,11.94,12.04,0.0
8987,2024-06-13,12.05,12.68,11.88,11.94,0.0
8988,2024-06-14,12.22,13.45,12.12,12.66,0.0
8989,2024-06-17,13.07,13.29,12.50,12.75,0.0
8990,2024-06-18,12.70,12.74,12.24,12.30,0.0
8992,2024-06-20,12.50,13.55,12.18,13.28,0.0
8993,2024-06-21,13.22,13.78,12.99,13.20,0.0


In [15]:
vix['10dayMA'] = vix['Close'].rolling(window=10).mean()
vix['percentAboveMA'] = 100*(vix['Close'] - vix['10dayMA']) / vix['10dayMA']
dates = vix[vix['percentAboveMA'] > 5].Date

In [16]:
spy = pd.read_csv('SPY.csv')
spy.dropna(inplace=True)
spy['Close'] = spy['Adj Close']
spy.drop(['Adj Close'], axis=1, inplace=True)

In [17]:
spy.tail(10)

,Date,Open,High,Low,Close,Volume
7895,2024-06-07,533.659973,536.890015,532.539978,532.292786,43224500
7896,2024-06-10,533.179993,535.989990,532.570007,533.937439,35729300
7897,2024-06-11,534.070007,537.010010,532.049988,535.223328,36383400
7898,2024-06-12,541.630005,544.119995,540.299988,539.619141,63251300
7899,2024-06-13,543.150024,543.330017,539.590027,540.705627,44760900
7900,2024-06-14,540.880005,542.809998,539.849976,541.034607,40089900
7901,2024-06-17,542.080017,548.530029,541.609985,545.340637,55839500
7902,2024-06-18,547.159973,548.619995,546.729980,546.726196,41376400
7903,2024-06-20,549.440002,550.119995,545.179993,545.240967,70328200
7904,2024-06-21,544.400024,545.650024,543.020020,544.510010,64338600


In [18]:
benchmark = []
for row in range(len(spy)-5):
    curr = spy.iloc[row].Close
    next = spy.iloc[row+5].Close
    diff = 100*(next - curr) / curr
    benchmark.append(diff)

In [19]:
dates = dates[:-2]

In [20]:
diffs = []
for date in dates:
    curr = spy[spy['Date'] == date].Close.values[0]
    currIndex = spy[spy['Date'] == date].index[0]
    nextIndex = currIndex + 5
    next = spy.iloc[nextIndex].Close
    diff = 100*(next - curr) / curr
    diffs.append(diff)

In [21]:
df = pd.DataFrame()
df['benchmark'] = benchmark
negEvents = diffs + (len(benchmark) - len(diffs))*[np.nan]
df['negEvents'] = negEvents
df.describe()

,benchmark,negEvents
count,7900.000000,1882.000000
mean,0.224128,0.419732
std,2.392196,2.958644
min,-19.793322,-19.793322
25%,-0.916129,-0.956842
50%,0.370485,0.596750
75%,1.517169,1.989790
max,19.403589,19.403589


In [22]:
t_statistic, p_value = ttest_ind(benchmark, diffs, equal_var=False)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

T-statistic: -2.6678680849937226
P-value: 0.007682730965259289
